In [1]:
import pandas as pd

# Specify the file path
file_path = 'D:/UNT/Data Vis/DataViZ Project/UI/New Branch/dataviz/cleaned_dataset/InternationalSalesReport/Cleaned_InternationalSalesReport.csv'
# Read the CSV file into a dataframe
df = pd.read_csv(file_path)
df


,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
0,0,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1.00,616.56,617.00
1,1,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1.00,616.56,617.00
2,2,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1.00,616.56,617.00
3,3,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1.00,616.56,617.00
4,4,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1.00,616.56,617.00
...,...,...,...,...,...,...,...,...,...,...
34953,37427,AVIN,03-31-22,Mar-22,PJNE3423,PJNE3423-KR-4XL,1.00,537.50,538.00,4.00
34954,37428,AVIN,03-31-22,Mar-22,PJNE3404,PJNE3404-KR-4XL,1.00,500.00,500.00,5.00
34955,37429,AVIN,03-31-22,Mar-22,PJNE3423,PJNE3423-KR-4XL,1.00,537.50,538.00,4.00
34956,37430,AVIN,03-31-22,Mar-22,SET290,SET290-KR-DPT-M,1.00,812.50,812.00,7.00


In [21]:
import pandas as pd
import json

# Load your DataFrame
# df = pd.read_csv('your_data.csv')

# Assuming 'cleaned_df' is your DataFrame containing the data

# Filter out rows with unexpected values in the "Months" column
cleaned_df = df[df['Months'].str.match(r'^[A-Za-z]{3}-\d{2}$')]

# Convert the "Months" column to datetime format
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Months'], format='%b-%y')

# Convert 'GROSS AMT' column to numeric format
cleaned_df['GROSS AMT'] = pd.to_numeric(cleaned_df['GROSS AMT'])

# Create a mapping dictionary for month abbreviations to numeric values
month_mapping = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# Map the month abbreviations to numeric values
cleaned_df['NumericMonth'] = cleaned_df['Date'].dt.strftime('%b').map(month_mapping)

# Sort the DataFrame by numeric month
cleaned_df = cleaned_df.sort_values('NumericMonth')

# Group the dataframe by numeric month and year, and sum up the gross amount
monthly_gross_amount = cleaned_df.groupby(['NumericMonth', cleaned_df['Date'].dt.year])['GROSS AMT'].sum().sort_index()

# Convert the grouped data to a dictionary with string keys
monthly_gross_dict = {f"{month}-{year}": float(gross_amount) for (month, year), gross_amount in monthly_gross_amount.items()}

# Get unique month names and sort them in ascending order
months_in_order = sorted(set(cleaned_df['NumericMonth']))

# Serialize the dictionary to a single JSON file
with open('monthly_gross_amount.json', 'w') as json_file:
    json.dump({'input': months_in_order, 'output': list(monthly_gross_dict.values())}, json_file, indent=4)


C:\Users\lokes\AppData\Local\Temp\ipykernel_29040\25834560.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Date'] = pd.to_datetime(cleaned_df['Months'], format='%b-%y')
C:\Users\lokes\AppData\Local\Temp\ipykernel_29040\25834560.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['GROSS AMT'] = pd.to_numeric(cleaned_df['GROSS AMT'])
C:\Users\lokes\AppData\Local\Temp\ipykernel_29040\25834560.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [10]:
import pandas as pd
import json

# Convert 'GROSS AMT' column to numeric format
df['GROSS AMT'] = pd.to_numeric(df['GROSS AMT'], errors='coerce')

# Group the dataframe by unique styles and sum the gross amount
df_grouped = df.groupby('Style')['GROSS AMT'].sum().reset_index()

# Sort the grouped dataframe by the sum of the gross amount in descending order
df_grouped_sorted = df_grouped.sort_values(by='GROSS AMT', ascending=False)

# Select the top 15 rows
top_15 = df_grouped_sorted.head(15)

# Convert the selected dataframe to a list of dictionaries
result = top_15.to_dict(orient='records')

# Dump the result to a JSON file
with open('top_15_style_grossamount.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)


In [12]:
import json


# Group by 'size' and calculate the sum of 'gross_amount'
df_grouped = df.groupby('Size')['GROSS AMT'].sum().reset_index()

# Filter only the specified sizes
valid_sizes = ['4XL', '5XL', '6XL', 'FREE', 'L', 'M', 'PCS', 'S', 'XL', 'XS', 'XXL', 'XXXL']
df_filtered = df_grouped[df_grouped['Size'].isin(valid_sizes)]

# Convert the filtered DataFrame to a list of dictionaries
result = [{'data': df_filtered.to_dict(orient='records')}]

# Dump the result to a JSON file
with open('sum_gross_amount_by_size.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)
